In [1]:
import os
import requests
import pandas as pd
import time 
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from datetime import datetime

In [2]:
trawlers = pd.read_csv('../raw_data/trawlers.csv')
trawlers['type']='trawlers'

In [3]:
longlines = pd.read_csv('../raw_data/drifting_longlines.csv')
longlines['type']='longlines'

In [4]:
gear = pd.read_csv('../raw_data/fixed_gear.csv')
gear['type']='gear'

In [5]:
pole = pd.read_csv('../raw_data/pole_and_line.csv')
pole['type']='pole'

In [6]:
seines = pd.read_csv('../raw_data/purse_seines.csv')
seines['type']='seines'

In [7]:
trollers = pd.read_csv('../raw_data/trollers.csv')
trollers['type']='trollers'

In [8]:
unknown = pd.read_csv('../raw_data/unknown.csv')
unknown['type']='unknown'

In [9]:
list_dfs = [trawlers, longlines, gear, pole, seines, trollers, unknown]
df = pd.concat(list_dfs)
df.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source,type
0,1.252340e+12,1.325376e+09,0.0,0.0,0.0,153.0,52.458649,4.581200,-1.0,gfw,trawlers
1,1.252340e+12,1.325378e+09,0.0,0.0,0.0,153.0,52.458668,4.581167,-1.0,gfw,trawlers
2,1.252340e+12,1.325379e+09,0.0,0.0,0.0,153.0,52.458633,4.581183,-1.0,gfw,trawlers


In [10]:
df['mmsi'].nunique()

354

In [11]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse_output =False)

# Fit encoder
ohe.fit(df[['type']])

# Transform the current "Alley" column
df[ohe.get_feature_names_out()] = ohe.transform(df[['type']])

# Drop the column "Alley" which has been encoded
df.drop(columns = ["type", "source"], inplace = True)

In [12]:
df.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,type_gear,type_longlines,type_pole,type_seines,type_trawlers,type_trollers,type_unknown
0,1.252340e+12,1.325376e+09,0.0,0.0,0.0,153.0,52.458649,4.581200,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.252340e+12,1.325378e+09,0.0,0.0,0.0,153.0,52.458668,4.581167,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.252340e+12,1.325379e+09,0.0,0.0,0.0,153.0,52.458633,4.581183,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [13]:
df_fishing = df.loc[df['is_fishing'] > -1]

In [14]:
df_fishing['is_fishing'].value_counts()

is_fishing
0.000000    295979
1.000000    247498
0.666667      4806
0.333333      4096
0.750000       752
0.250000       670
0.800000        33
0.166667        12
0.400000         9
Name: count, dtype: int64

In [15]:
df_fishing.loc[:, ('is_fishing')] = round(df_fishing.loc[:, ('is_fishing')])

In [16]:
df_fishing['is_fishing'].value_counts()

is_fishing
0.0    300766
1.0    253089
Name: count, dtype: int64

In [17]:
df_fishing.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,type_gear,type_longlines,type_pole,type_seines,type_trawlers,type_trollers,type_unknown
60646,1.252340e+12,1.420089e+09,0.0,0.0,0.0,128.0,52.458717,4.581316,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
60647,1.252340e+12,1.420090e+09,0.0,0.0,0.0,128.0,52.458733,4.581316,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
60648,1.252340e+12,1.420090e+09,0.0,0.0,0.0,128.0,52.458698,4.581267,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [18]:
df_fishing['timestamp'] = pd.to_datetime(df_fishing['timestamp'], unit='s')

/var/folders/jp/3hlxbh4x1mb0k2qwf5fmzyp80000gp/T/ipykernel_81408/4072986675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fishing['timestamp'] = pd.to_datetime(df_fishing['timestamp'], unit='s')


In [19]:
df_fishing.head(2)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,type_gear,type_longlines,type_pole,type_seines,type_trawlers,type_trollers,type_unknown
60646,1.252340e+12,2015-01-01 05:08:23,0.0,0.0,0.0,128.0,52.458717,4.581316,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
60647,1.252340e+12,2015-01-01 05:20:34,0.0,0.0,0.0,128.0,52.458733,4.581316,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [20]:
df_fishing.rename(columns={"timestamp": "date"}, inplace=True)
df_fishing.head()

/var/folders/jp/3hlxbh4x1mb0k2qwf5fmzyp80000gp/T/ipykernel_81408/4110272914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fishing.rename(columns={"timestamp": "date"}, inplace=True)


,mmsi,date,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,type_gear,type_longlines,type_pole,type_seines,type_trawlers,type_trollers,type_unknown
60646,1.252340e+12,2015-01-01 05:08:23,0.0,0.0,0.0,128.0,52.458717,4.581316,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
60647,1.252340e+12,2015-01-01 05:20:34,0.0,0.0,0.0,128.0,52.458733,4.581316,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
60648,1.252340e+12,2015-01-01 05:32:53,0.0,0.0,0.0,128.0,52.458698,4.581267,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
60649,1.252340e+12,2015-01-01 05:45:23,0.0,0.0,0.0,128.0,52.458698,4.581234,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
60650,1.252340e+12,2015-01-01 05:57:24,0.0,0.0,0.0,128.0,52.458683,4.581183,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
df_fishing['year'] = df_fishing['date'].dt.year
df_fishing['month'] = df_fishing['date'].dt.month
df_fishing['day'] = df_fishing['date'].dt.day
df_fishing['day_of_week'] = df_fishing['date'].dt.day_of_week
df_fishing['day_of_year'] = df_fishing['date'].dt.day_of_year

/var/folders/jp/3hlxbh4x1mb0k2qwf5fmzyp80000gp/T/ipykernel_81408/229382827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fishing['year'] = df_fishing['date'].dt.year
/var/folders/jp/3hlxbh4x1mb0k2qwf5fmzyp80000gp/T/ipykernel_81408/229382827.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fishing['month'] = df_fishing['date'].dt.month
/var/folders/jp/3hlxbh4x1mb0k2qwf5fmzyp80000gp/T/ipykernel_81408/229382827.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [22]:
df_fishing.head()

,mmsi,date,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,type_gear,...,type_pole,type_seines,type_trawlers,type_trollers,type_unknown,year,month,day,day_of_week,day_of_year
60646,1.252340e+12,2015-01-01 05:08:23,0.0,0.0,0.0,128.0,52.458717,4.581316,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2015,1,1,3,1
60647,1.252340e+12,2015-01-01 05:20:34,0.0,0.0,0.0,128.0,52.458733,4.581316,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2015,1,1,3,1
60648,1.252340e+12,2015-01-01 05:32:53,0.0,0.0,0.0,128.0,52.458698,4.581267,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2015,1,1,3,1
60649,1.252340e+12,2015-01-01 05:45:23,0.0,0.0,0.0,128.0,52.458698,4.581234,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2015,1,1,3,1
60650,1.252340e+12,2015-01-01 05:57:24,0.0,0.0,0.0,128.0,52.458683,4.581183,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2015,1,1,3,1


In [ ]:
import pytz
from datetime import datetime

def get_utc_offset_from_longitude(longitude):
    timezone = pytz.timezone(pytz.country_timezones("US")[0])  # You can replace "US" with the appropriate country code
    now = datetime.now(timezone)
    utc_offset = now.utcoffset().total_seconds() / 3600
    return utc_offset

df_fishing['utc_offset'] = df_fishing['lon'].apply(get_utc_offset_from_longitude)
longitude = -75.1652  # Replace with your longitude
utc_offset = get_utc_offset_from_longitude(longitude)
print(f"UTC Offset for longitude {longitude}: {utc_offset} hours")